It used to be that employees had to file expense reports by hand, or at least draft them on a spreadsheet. Then computers came along, with built-in spreadsheets that made it easier to generate and track reports. But what if you could put all of your personnel through training on how to use such software and automate the reports? 

In this article I will show you the steps to automate expense reports using Python.

---

# Steps

The steps to automate expense reports will be:

1. Create an account on the [mindee](https://platform.mindee.com/) platform
2. Set up an API key
3. Install the "mindee" package
4. Import dependencies
5. Write helper functions
6. Load, Parse and Extract the Data from the Expense Receipts 
7. Export Results to a Table
8. Save Table to `.csv` File

Let's get started!

---

## 1. Create an Account on the [mindee](https://platform.mindee.com/) Platform

For this automation in order to avoid having to write custom code for detecting the text in the images of the receipts, what we will do is use the `mindee` package which comes with an API that allows you to do all of that with only a few lines of code. Although a professional version of this package is paid, they offer 250 pages a month for free, which for individuals should be more than enough to automate their personal expense reports. 

To create the account do the following:

1. Head over to the [`mindee` platform website](https://platform.mindee.com/)
2. Sign up

## 2. Set up an API Key

To set your API key just do the following:

1. Click on "Create a new API"
2. Select the "Expense Receipt" API
3. Copy your API key and save it.

## 3. Install the 'mindee' Package

To install the `mindee` package run:

```
pip install mindee
```

## 4. Import Dependencies

For this project we will be using the following packages: 
- `mindee`
- `pandas`
- `random`
- `glob`
- `matplotlib`
- `seaborn`

If you dont have them in your local environment, install them with `pip install <package>`.

Now we can import our dependencies:


In [1]:
from mindee import Client, documents
import random
import pandas as pd
import glob
# Sanity Check using pandas and matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


## 5. Write Helper Functions 


For this automation we will need 3 helper functions: one for extracting the 
expense data after getting the response frmo the `mindee` API, another for 
converting time to meal type (in this example my expense reports involve having 
to explicit lunch or dinner for food expenses) and finally another function
for creating the final table with all of our data.

The code will be:

In [2]:
# Helper functions to extract the information I want
def extract_expenses_data(api_response):
    """
    Extracts the data from the response returned by the 
    mindee API. In this case we extract, the data of the receipt,
    the category (like food, transport, etc..) the time of the processing of the
    receipt as well as the total amount and the filename of the image. 
    """
    date = api_response.date.value
    category = api_response.category.value
    time = api_response.time.value
    amount = api_response.total_amount.value
    filename = api_response.filename
    
    return date, amount, filename, category, time

def convert_time_to_meal_type(time, hour_max = 17, hour_min=6):
    """
    This function takes as input a string `time` and returns
    whether that time slot corresponds to dinner or lunch given a threshold
    variable separating dinner and lunch times.
    """
    hour_digits = time[:2]
    if int(hour_digits)<hour_max and int(hour_digits)>hour_min:
        meal_type = "Lunch"
    else:
        meal_type = "Dinner"
    
    return meal_type
    

# Code for testing the `convert_time_to_meal_type()` function.
hours_for_test = [str(random.randint(10,23)).zfill(2) + ":" + str(random.randint(0,59)).zfill(2) for i in range(10)]
print(hours_for_test)
for hour in  hours_for_test:
    print(hour, convert_time_to_meal_type(hour))


def create_table(date_list,amount_list,meal_type, time_list,category_list, filenames_list):
    """
    This functions takes as input 6 lists for date, amount, meal type, time,
    category and filename. It returns a pandas dataframe with the data
    structured as a table.
    """
    df = pd.DataFrame(
        {"date":date_list, "amount":amount_list, "meal_type":meal_type, 
         "time":time_list, "category":category_list, "filename":filenames_list}
    )
    return df

# Testing the `create_table` function
create_table(["2021-01-01","2021-01-02"],[10,20],["Lunch","Dinner"],["10:00","20:00"],["Food","Food"],["file1","file2"])

['13:51', '11:49', '22:13', '19:57', '10:32', '20:47', '20:40', '14:27', '14:41', '15:06']
13:51 Lunch
11:49 Lunch
22:13 Dinner
19:57 Dinner
10:32 Lunch
20:47 Dinner
20:40 Dinner
14:27 Lunch
14:41 Lunch
15:06 Lunch


,date,amount,meal_type,time,category,filename
0,2021-01-01,10,Lunch,10:00,Food,file1
1,2021-01-02,20,Dinner,20:00,Food,file2


## 6. Load, Parse and Extract the Data from the Expense Receipts

Now, all we have to do is:

1. **Instantiate our *mindee* client using the API key we obtained**

In [3]:
# Instantiate a new client
mindee_client = Client(api_key="Your API KEY")

2. **Initialize some empty lists that will contain the data extracted**


In [4]:
date_list = []
amount_list = []
category_list = []
time_list = []
meal_type_list = []
filenames_list = []

3. **Load image of expense receipt and feed it to the mindee api**

In [7]:
image = "./expense_images/1669895159779.jpg"
input_doc = mindee_client.doc_from_path(image)
api_response = input_doc.parse(documents.TypeReceiptV4)
expense_data = api_response.document
expense_data

The output will be a mindee object that is tailored for expense receipts
(there are probably a few options so feel free to investigate that in the official
documentation in the mindee platform).

4. **Extract the expenses information from the api response**

In [8]:
date, amount, filename, category, time = extract_expenses_data(expense_data)

5. **Convert the time of day information into relevant meal type information**

This example is very specific to my particular case, so you might change this 
function according to the types of expenses you have. But here, what I am doing
is transforming a string like `13:30` to lunch and a string like `20:30`
to dinner.

In [11]:
if not time:
        meal_type = "Unknown"
else:
    meal_type = convert_time_to_meal_type(time)

6. **Append the extracted information to their corresopnding lists**

In this case I am only doing for a single receipt, but when doing for
multiple receipts the list approach will make more sense.

In [12]:
date_list.append(date)
# I am replacing the . for , here because the final report goes on
# a google sheet which takes `,` instead of `.` for float numbers.
amount_list.append(str(amount).replace(".", ","))
category_list.append(category)
time_list.append(time)
meal_type_list.append(meal_type)
filenames_list.append(filename)

## 7. Export Results to a Table

Now that we have the data organized, we can create a table with the structured 
data.

In [13]:
# The format of the table will be | data | amount | filename | category |
df = create_table(date_list,amount_list,meal_type_list, time_list,category_list, 
                  filenames_list)

# show the table to check
df

,date,amount,meal_type,time,category,filename
0,2022-11-26,"32,25",Dinner,23:35,food,1669895159779.jpg


## 8. Save Table to `.csv` File

Now to finish things off we save that table with all the information we
wanted from the expense receipts.

In [ ]:
df.to_csv("expense_table.csv", header=False)

---

# Bringing Everything Together

Now that we know each step in detail, the entire source code for running this 
on multiple expense receipts:

In [20]:
# Run the main loop for the automation

# Instantiate a new client
mindee_client = Client(api_key="Your API KEY")

date_list = []
amount_list = []
category_list = []
time_list = []
meal_type_list = []
filenames_list = []
for image in glob.glob("./expense_images/*"):
    input_doc = mindee_client.doc_from_path(image)
    api_response = input_doc.parse(documents.TypeReceiptV4)
    expense_data = api_response.document
    date, amount, filename, category, time = extract_expenses_data(expense_data)
    if not time:
        meal_type = "Unknown"
    else:
        meal_type = convert_time_to_meal_type(time)
    
    date_list.append(date)
    amount_list.append(str(amount).replace(".", ","))
    category_list.append(category)
    time_list.append(time)
    meal_type_list.append(meal_type)
    filenames_list.append(filename)


# the model of the final table will be:

# | data | valor | filename | category |
df = create_table(date_list,amount_list,meal_type_list, time_list,category_list, 
                  filenames_list)

# sort values in the table by meal type (specific to my use case)
df.sort_values(by="meal_type").to_csv("expense_table.csv", header=False)

# show the table to check
df

,date,amount,meal_type,time,category,filename
0,2022-11-10,"24,9",Lunch,14:19,food,1669895159846.jpg
1,2022-11-08,"18,55",Dinner,20:15,food,1669895159855.jpg
2,2022-11-26,"32,25",Dinner,23:35,food,1669895159779.jpg
3,2022-11-10,"86,75",Unknown,None,food,1669895159837.jpg
4,2022-10-29,"41,73",Dinner,18:37,food,1669895159872.jpg
5,2022-10-30,"23,9",Lunch,12:59,food,1669895159864.jpg
6,2022-11-24,"11,55",Dinner,20:28,food,1669895159808.jpg
7,2022-10-23,"9,4",Dinner,17:00,food,1669895159908.jpg
8,2022-11-25,"14,6",Dinner,20:11,food,1669895159796.jpg
9,2022-10-27,"12,0",Lunch,16:22,food,1669895159880.jpg


THere you have it! You automated the boring task of reporting your expenses
from images of receipts! As a final check, it's always good to take a look at the 
final results to make sure the information you are getting is consistent with the
actual data in the expense receipts. For that we can visualize the receipts,
side by side with the text data obtained from the extraction for each expense
receipt using matplotlib.

In [14]:
# Loop over the filenames and visualize the images side by side with the
# data extracted
for i,filename in enumerate(df.filename):
    image = plt.imread(f"./expense_images/{filename}")
    plt.figure(figsize=(15,10))
    plt.imshow(image)
    date = df[df["filename"]==filename]["date"]
    amount = df[df["filename"]==filename]["amount"]
    meal_type = df[df["filename"]==filename]["meal_type"]
    time = df[df["filename"]==filename]["time"]
    plt.title(f"date: {date} amount: {amount} meal_type: {meal_type} time: {time}")
    plt.show()
    input("Continue?")

*I will be showing just a couple of images with restricted information for privacy reasons
but the overall idea is here*.

Ok, the results seem consistent! There we have it, a neat automation to save you
some time every month!

---

# Final Thoughts on Automating Everything

If you're a manager, developer, accountant or pretty much anyone who works with data entry of some kind, this approach  may be very useful to you. For one thing, it clearly shows that automation in the workplace is possible with just a few lines of Python code. 

Why not use a simple program to automate your expense report processing? This way you can save time and eliminate human error. Automating repetitive tasks like these can save you quite a bit of time and it opens the door to streamline your entire workflow.